In [1]:
# For data_analysis env

import os
import tkinter.filedialog as fdialog #window to select directories
from tqdm import tqdm #progress bars

import shutil
import time

#import anne_code.Neuron_analysis as na #custum functions
#from anne_code.Neuron_analysis import *

import re
import numpy as np
import numpy.ma as ma
import pandas as pd
import matplotlib.pyplot as plt

import SimpleITK as sitk #elastix
import skimage
from skimage import io
from skimage import morphology
from skimage import measure
import cv2
from PIL import Image, ImageChops

import warnings
import plotly.graph_objects as go

import dask_image.imread
import random

from skelly_module import *

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
seg_al215_20220329_1_path = '/data/Lucas/AL215/seg-Al215_20220329_1'
seg_al215_20220330_1_path = '/data/Lucas/AL215/seg-Al215_20220330_1'
seg_al215_default_path = '/data/Lucas/AL215/seg-Al215_default'
seg_al215_default_dense_path = '/data/Lucas/AL215/seg-Al215_default_dense'

In [4]:
seg_al215_20220329_1 = dask_image.imread.imread(seg_al215_20220329_1_path + '/*.tif')
seg_al215_20220330_1 = dask_image.imread.imread(seg_al215_20220330_1_path + '/*.tif')
seg_al215_default = dask_image.imread.imread(seg_al215_default_path + '/*.tif')
seg_al215_default_dense = dask_image.imread.imread(seg_al215_default_dense_path + '/*.tif')

In [60]:
diff_s, m1, m2, diff_n = [], [], [], []
n_pixels = seg_al215_20220329_1.shape[1]*seg_al215_20220329_1.shape[2]
for _ in range(40):
    r_i = random.randint(0,seg_al215_20220329_1.shape[0])
    img1 = seg_al215_20220329_1[r_i].compute()
    img2 = seg_al215_20220330_1[r_i].compute()
    m1.append(np.mean(img1[img1>0]))
    m2.append(np.mean(img2[img2>0]))
    diff_img = np.abs(img1-img2)
    diff_s.append(np.mean(diff_img))
    diff_n.append((len(img1[img1>0])-len(img2[img2>0]))/n_pixels)
    
print("Mean difference between two stacks (value):", np.mean(diff_s))
print("Mean difference between two stacks (%non-zero pixels):", np.mean(diff_n)*100)
print("Mean pixel value of img1 (non-zero):", np.mean(m1))
print("Mean pixel value of img2 (non-zero):", np.mean(m2))

**Reload kernel before next**

In [5]:
%%time
def binarize_stack(dask_stack):
    #seg1 = dask_stack.astype('float32')
    dask_stack[dask_stack>= 0.3] = 1
    dask_stack[dask_stack< 0.3] = 0
    return dask_stack.astype('uint8').compute()
    
bin_s = binarize_stack(seg_al215_20220329_1)

CPU times: user 37.9 s, sys: 54.9 s, total: 1min 32s
Wall time: 1min 20s


In [7]:
bin_s.shape

(1359, 2048, 2048)

In [13]:
bin_s.shape

(1359, 2048, 2048)

In [14]:
bin_s[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [6]:
%%time
list_files=get_dir(seg_al215_20220330_1_path)
img_stack = [None]*len(list_files)
for count, item in enumerate(list_files):
    img = cv2.imread(item, cv2.COLOR_BGR2GRAY)
    img[img >= 0.3] = 1
    img[img < 0.3] = 0
    img_stack[count] = img
    
img_stack = np.array(img_stack, dtype=np.uint8)

CPU times: user 36.2 s, sys: 25.7 s, total: 1min 1s
Wall time: 1min 39s


In [10]:
list_files[1]

'seg-1000_AL215_rLS.tif'

**New skeleton**

In [5]:
def skeletons_3d_sep(input_folder, output_folder_loc):
    
    skel_count = 0
    folder_list = []
    thresh_list = 0.1*np.array(range(2,10,1))
            
    output_name = "skel-" + os.path.basename(input_folder)
    output_dir = os.path.dirname(output_folder_loc)
    output_folder = os.path.join(output_dir, output_name)
    if os.path.exists(output_folder):
        print(output_folder + " already exists. Will be overwritten.")
        shutil.rmtree(output_folder)
    os.makedirs(output_folder) 
    
    for thresh in thresh_list:
        dask_stack = dask_image.imread.imread(input_folder + "/*.tif")
        dask_stack[dask_stack>=thresh] = 1
        dask_stack[dask_stack<thresh] = 0
        img_stack = dask_stack.astype('uint8').compute()
        img_stack = morphology.skeletonize_3d(img_stack) # max output from skeletonize is 255 (0->255), uint8 data type
        print("Skeleton for threshold", thresh, "computed !")
        folder_save = output_folder + '/skel_thresh_' + str(skel_count)
        if not os.path.exists(folder_save):
            os.makedirs(folder_save)
            print(folder_save, "directory created !")
        for count in range(dask_stack.shape[0]):
            pil_image = Image.fromarray(img_stack[count])
            pil_image.save(folder_save + '/' + os.path.basename("skeleton_image_" + str(count) + ".tif"))
        print("Images for threshold", thresh, "saved !")
        skel_count += 1
        folder_list.append(folder_save)
        
    return folder_list

In [6]:
skeletons_3d_dask(seg_al215_20220330_1_path, "/data/Lucas/AL215")

Skeleton for threshold 0.2 computed !
Images for threshold 0.2 saved !
Skeleton for threshold 0.30000000000000004 computed !
Images for threshold 0.30000000000000004 saved !
Skeleton for threshold 0.4 computed !
Images for threshold 0.4 saved !
Skeleton for threshold 0.5 computed !
/data/Lucas/AL215/skel_thresh_3 directory created !
Images for threshold 0.5 saved !
Skeleton for threshold 0.6 computed !
/data/Lucas/AL215/skel_thresh_4 directory created !
Images for threshold 0.6 saved !
Skeleton for threshold 0.7 computed !
/data/Lucas/AL215/skel_thresh_5 directory created !
Images for threshold 0.7 saved !
Skeleton for threshold 0.7999999999999999 computed !
/data/Lucas/AL215/skel_thresh_6 directory created !
Images for threshold 0.7999999999999999 saved !
Skeleton for threshold 0.8999999999999999 computed !
/data/Lucas/AL215/skel_thresh_7 directory created !
Images for threshold 0.8999999999999999 saved !


['/data/Lucas/AL215/skel_thresh_0',
 '/data/Lucas/AL215/skel_thresh_1',
 '/data/Lucas/AL215/skel_thresh_2',
 '/data/Lucas/AL215/skel_thresh_3',
 '/data/Lucas/AL215/skel_thresh_4',
 '/data/Lucas/AL215/skel_thresh_5',
 '/data/Lucas/AL215/skel_thresh_6',
 '/data/Lucas/AL215/skel_thresh_7']

In [10]:
a = np.zeros((100,200,300))

In [12]:
a[:].shape

(100, 200, 300)

**Check skeletons**

In [10]:
skel0 = dask_image.imread.imread(r'/data/Lucas/AL215/skel_thresh_0/*.tif')
skel3 = dask_image.imread.imread(r'/data/Lucas/AL215/skel_thresh_3/*.tif')
skel5 = dask_image.imread.imread(r'/data/Lucas/AL215/skel_thresh_5/*.tif')
skel7 = dask_image.imread.imread(r'/data/Lucas/AL215/skel_thresh_7/*.tif')

In [12]:
np.mean(skel0.compute())

0.0011911411509959464

In [14]:
np.mean(skel3.compute())

0.00099400845935364

In [15]:
np.mean(skel5.compute())

0.0008979517366186147

In [13]:
np.mean(skel7.compute())

0.0007705351637951849

**Weighted sum**

In [46]:
list_files=get_dir(r'/data/Lucas/AL215/skel_thresh_0/')

In [47]:
%%time
img1 = skel0[0].compute()

CPU times: user 21.5 ms, sys: 153 µs, total: 21.6 ms
Wall time: 21 ms


In [48]:
%%time
img2 = cv2.imread(list_files[0], cv2.COLOR_BGR2GRAY)

CPU times: user 14.1 ms, sys: 229 µs, total: 14.4 ms
Wall time: 13.4 ms


In [56]:
folder_list = ['/data/Lucas/AL215/skel_thresh_0',
 '/data/Lucas/AL215/skel_thresh_1',
 '/data/Lucas/AL215/skel_thresh_2',
 '/data/Lucas/AL215/skel_thresh_3',
 '/data/Lucas/AL215/skel_thresh_4',
 '/data/Lucas/AL215/skel_thresh_5',
 '/data/Lucas/AL215/skel_thresh_6',
 '/data/Lucas/AL215/skel_thresh_7']

In [48]:
list_files=get_dir(folder_list[0])
thresh_list = 0.1*np.array(range(2,10,1))
img_f = np.multiply(thresh_list[0],cv2.imread(folder + "/" + os.path.basename(list_files[0]), cv2.COLOR_BGR2GRAY))
folder_save = r'/data/Lucas/AL215/'
for count, folder in enumerate(folder_list[1:]):
    img_f = np.add(img_f, np.multiply(thresh_list[count+1],cv2.imread(folder + "/" + os.path.basename(list_files[0]), cv2.COLOR_BGR2GRAY)))
    pil_img = Image.fromarray(img_f)
    pil_img.save(folder_save + '/def-' + os.path.basename(list_files[0]))

In [5]:
def skeletons_3d(input_folder, output_folder_loc):
    
    skel_count = 0
    folder_list = []
    thresh_list = 0.1*np.array(range(2,10,1))
            
    output_name = "skel-" + os.path.basename(input_folder)
    output_folder = os.path.join(output_folder_loc, output_name)
    if os.path.exists(output_folder):
        print(output_folder + " already exists. Will be overwritten.")
        shutil.rmtree(output_folder)
    os.makedirs(output_folder) 
    
    for thresh in thresh_list:
        dask_stack = dask_image.imread.imread(input_folder + "/*.tif")
        dask_stack[dask_stack>=thresh] = 1
        dask_stack[dask_stack<thresh] = 0
        img_stack = dask_stack.astype('uint8').compute()
        img_stack = morphology.skeletonize_3d(img_stack) # max output from skeletonize is 255 (0->255), uint8 data type
        print("Skeleton for threshold", thresh, "computed !")
        folder_save = output_folder + '/skel_thresh_' + str(skel_count)
        if not os.path.exists(folder_save):
            os.makedirs(folder_save)
            print(folder_save, "directory created !")
        for count in range(dask_stack.shape[0]):
            pil_image = Image.fromarray(img_stack[count])
            pil_image.save(folder_save + '/' + os.path.basename("skeleton_image_" + str(count + 100000) + ".tif"))
        print("Images for threshold", thresh, "saved !")
        skel_count += 1
        folder_list.append(folder_save)
        
    list_files=get_dir(folder_list[0])
    folder_save = output_folder + "/weighted_sum_skeleton"
    if not os.path.exists(folder_save):
        os.makedirs(folder_save)
        print(folder_save, "directory created !")
    for i in range(tqdm(len(list_files))):
        img_f = np.multiply(thresh_list[0],cv2.imread(folder_list[0] + "/" + os.path.basename(list_files[i]), cv2.COLOR_BGR2GRAY))
        for count, folder in enumerate(folder_list[1:]):
            img_f = np.add(img_f, np.multiply(thresh_list[count+1],cv2.imread(folder + "/" + os.path.basename(list_files[i]), cv2.COLOR_BGR2GRAY)))
            pil_img = Image.fromarray(img_f)
            pil_img.save(folder_save + '/def-' + os.path.basename(list_files[i]))
        
    return folder_list

In [8]:
skeletons_3d('/data/Lucas/AL207/seg-Al207_20220329_1', "/data/Lucas/AL207")

Skeleton for threshold 0.2 computed !
/data/Lucas/AL207/skel-seg-Al207_20220329_1/skel_thresh_0 directory created !
Images for threshold 0.2 saved !
Skeleton for threshold 0.30000000000000004 computed !
/data/Lucas/AL207/skel-seg-Al207_20220329_1/skel_thresh_1 directory created !
Images for threshold 0.30000000000000004 saved !
Skeleton for threshold 0.4 computed !
/data/Lucas/AL207/skel-seg-Al207_20220329_1/skel_thresh_2 directory created !
Images for threshold 0.4 saved !
Skeleton for threshold 0.5 computed !
/data/Lucas/AL207/skel-seg-Al207_20220329_1/skel_thresh_3 directory created !
Images for threshold 0.5 saved !
Skeleton for threshold 0.6000000000000001 computed !
/data/Lucas/AL207/skel-seg-Al207_20220329_1/skel_thresh_4 directory created !
Images for threshold 0.6000000000000001 saved !
Skeleton for threshold 0.7000000000000001 computed !
/data/Lucas/AL207/skel-seg-Al207_20220329_1/skel_thresh_5 directory created !
Images for threshold 0.7000000000000001 saved !
Skeleton for t

['/data/Lucas/AL207/skel-seg-Al207_20220329_1/skel_thresh_0',
 '/data/Lucas/AL207/skel-seg-Al207_20220329_1/skel_thresh_1',
 '/data/Lucas/AL207/skel-seg-Al207_20220329_1/skel_thresh_2',
 '/data/Lucas/AL207/skel-seg-Al207_20220329_1/skel_thresh_3',
 '/data/Lucas/AL207/skel-seg-Al207_20220329_1/skel_thresh_4',
 '/data/Lucas/AL207/skel-seg-Al207_20220329_1/skel_thresh_5',
 '/data/Lucas/AL207/skel-seg-Al207_20220329_1/skel_thresh_6',
 '/data/Lucas/AL207/skel-seg-Al207_20220329_1/skel_thresh_7']

In [61]:
%load_ext Cython

In [67]:
%%cython
from PIL import Image
import cv2
import numpy as np
import os
from skelly_module import *

def weighted_sum_skeleton(folder_list, thresh_list, output_folder):
    if len(folder_list) != len(thresh_list):
        print('Error ! Input of different sizes.)')
        return
    list_files=get_dir(folder_list[0])
    folder_save = output_folder + "/weighted_sum_skeleton"
    if not os.path.exists(folder_save):
        os.makedirs(folder_save)
        print(folder_save, "directory created !")
    for i in range(len(list_files)):
        img_f = np.multiply(thresh_list[0],cv2.imread(folder_list[0] + "/" + os.path.basename(list_files[i]), cv2.COLOR_BGR2GRAY))
        for count, folder in enumerate(folder_list[1:]):
            img_f = np.add(img_f, np.multiply(thresh_list[count+1],cv2.imread(folder + "/" + os.path.basename(list_files[i]), cv2.COLOR_BGR2GRAY)))
            pil_img = Image.fromarray(img_f)
            pil_img.save(folder_save + '/def-' + os.path.basename(list_files[i]))

In [57]:
folder_list = ['/data/Lucas/AL207/skel-seg-Al207-substack/skel_thresh_0',
 '/data/Lucas/AL207/skel-seg-Al207-substack/skel_thresh_1',
 '/data/Lucas/AL207/skel-seg-Al207-substack/skel_thresh_2',
 '/data/Lucas/AL207/skel-seg-Al207-substack/skel_thresh_3',
 '/data/Lucas/AL207/skel-seg-Al207-substack/skel_thresh_4',
 '/data/Lucas/AL207/skel-seg-Al207-substack/skel_thresh_5',
 '/data/Lucas/AL207/skel-seg-Al207-substack/skel_thresh_6',
 '/data/Lucas/AL207/skel-seg-Al207-substack/skel_thresh_7']

thresh_list = 0.1*np.array(range(2,10,1))
output_folder = r'/data/Lucas/AL207/skel-seg-Al207-substack'

In [68]:
weighted_sum_skeleton(folder_list, thresh_list, output_folder)

### Better skeleton testing

Might not have time to implement <br/>
missing ref

In [13]:
import numpy as np
import time
from scipy import ndimage
from scipy.ndimage.filters import convolve

In [14]:
"""
   the following subiteration functions are how each image is rotated to the next direction for removing
   boundary voxels in the order described in the reference paper
   us, ne, wd,..
   
    reference paper
   http://web.inf.u-szeged.hu/ipcg/publications/papers/PalagyiKuba_GMIP1999.pdf
   input should be a binary image/ already segmented
"""
from rotationalOperators.rotational_operators import get_directions_list

In [15]:
directions_list = get_directions_list(np.zeros((3,3,3)))

In [16]:
"""
   array that has calculated the validity of the 14 templates beforehand and stored each index which is
   decimal number of the binary string of 26 values (sqrt(3) connectivity) that are around a single voxel 
"""
lookUpTablearray = np.load('rotationalOperators/lookupTablearray.npy')

In [17]:
def _convolveImage(arr, flippedKernel):
    arr = np.ascontiguousarray(arr, dtype=np.uint64)
    result = convolve(arr, flippedKernel, mode='constant', cval=0)
    result[arr == 0] = 0
    return result

sElement = ndimage.generate_binary_structure(3, 1)

def _getBouondariesOfimage(image):
    """
       function to find boundaries/border/edges of the array/image
    """

    erode_im = ndimage.morphology.binary_erosion(image, sElement)
    boundaryIm = image - erode_im
    return boundaryIm

def _skeletonPass(image):
    """
        each pass consists of 12 serial subiterations and finding the
        boundaries of the padded image/array
    """
    boundaryIm = _getBouondariesOfimage(image)
    numPixelsremovedList = [] * 12
    boundaryIndices = list(set(map(tuple, list(np.transpose(np.nonzero(boundaryIm))))))
    directionList = get_directions_list(image)
    for i in range(len(directionList)):
        convImage = _convolveImage(image, directionList[i])
        totalPixels, image = _applySubiter(image, boundaryIndices, convImage)
        print("number of pixels removed in the {} direction is {}". format(i, totalPixels))
        numPixelsremovedList.append(totalPixels)
    numPixelsremoved = sum(numPixelsremovedList)
    return numPixelsremoved, image


def _applySubiter(image, boundaryIndices, convImage):
    """
       each subiteration paralleley reduces the border voxels in 12 directions
       going through each voxel and marking if it can be deleted or not in a
       different image named temp_del and finally multiply it with the original
       image to delete the voxels so marked
    """
    temp_del = np.zeros_like(image)
    # boundaryIndicesCopy = copy.deepcopy(boundaryIndices)
    lenB = len(boundaryIndices)
    for k in range(0, lenB):
        temp_del[boundaryIndices[k]] = lookUpTablearray[convImage[boundaryIndices[k]]]
    numpixel_removed = np.einsum('ijk->', image * temp_del, dtype=int)
    image[temp_del == 1] = 0
    return numpixel_removed, image


def getSkeletonize3D(image):
    """
    function to skeletonize a 3D binary image with object in brighter contrast than background.
    In other words, 1 = object, 0 = background
    """
    assert np.max(image) in [0, 1]
    zOrig, yOrig, xOrig = np.shape(image)
    padImage = np.lib.pad(image, 1, 'constant', constant_values=0)
    start_skeleton = time.time()
    pass_no = 0
    numpixel_removed = 0
    while pass_no == 0 or numpixel_removed > 0:
        numpixel_removed, padImage = _skeletonPass(padImage)
        print("number of pixels removed in pass {} is {}".format(pass_no, numpixel_removed))
        pass_no += 1
    print("done %i number of pixels in %f seconds" % (np.sum(image), time.time() - start_skeleton))
    return padImage[1: zOrig + 1, 1: yOrig + 1, 1: xOrig + 1]

In [20]:
sample = np.ones((5, 5, 6), dtype=np.uint8)
resultSkel = getSkeletonize3D(sample)

number of pixels removed in the 0 direction is 0
number of pixels removed in the 1 direction is 0
number of pixels removed in the 2 direction is 0
number of pixels removed in the 3 direction is 0
number of pixels removed in the 4 direction is 0
number of pixels removed in the 5 direction is 0
number of pixels removed in the 6 direction is 0
number of pixels removed in the 7 direction is 0
number of pixels removed in the 8 direction is 0
number of pixels removed in the 9 direction is 0
number of pixels removed in the 10 direction is 0
number of pixels removed in the 11 direction is 0
number of pixels removed in pass 0 is 0
done 150 number of pixels in 0.031449 seconds


In [19]:
resultSkel

array([[[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]],

       [[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]],

       [[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]],

       [[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]],

       [[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]]], dtype=uint8)

### Small skeletons

In [ ]:
%load_ext Cython

In [ ]:
import cc3d

In [ ]:
%%cython
import numpy as np
cimport numpy as np

def set_to_zero(np.ndarray[float, ndim=3] vol, np.ndarray[long long, ndim=2] list_coords):
    cdef int i = 0
    for i in range(len(list_coords)):
        vol[list_coords[i][0],list_coords[i][1],list_coords[i][2]] = 0
    
    return vol

In [ ]:
def small_skeletons_3d(img_stack):
    
    thresh_list = 0.1*np.array(range(2,10,1))
    skel_stack = np.zeros(img_stack.shape, dtype=np.float32)
    
    for thresh in thresh_list:
        binary_stack = np.copy(img_stack.view(dtype=np.float32)) #forces a copy, actually segmentation output is already float32 from TrailMap
        binary_stack[binary_stack>=thresh] = 1
        binary_stack[binary_stack<thresh] = 0
        binary_stack = morphology.skeletonize_3d(binary_stack)
        skel_stack = np.add(skel_stack, np.multiply(binary_stack, thresh, dtype=np.float32), dtype=np.float32)
        print("Skeleton for threshold", thresh, "computed !")
        
    return skel_stack

In [ ]:
img_stack = from_folder_to_stack(r'/data/Lucas/Substacks/AL215s/seg-Al215-substack-20220505-6')

In [ ]:
def trim_small_elements(stack, size_threshold):
    #Removing small elements increases the mean probability of remaining pixels.
    #(Small elements are associated with lower probabilities of axon presence)
    
    label_stack = np.copy(stack.view(dtype=np.float32)) # force a copy
    label_stack[label_stack > 0] = 1
    label_stack = measure.label(label_stack, background=0) #computationally expensive
    print('Start: individual "blobs":', np.max(label_stack))
    
    propsa = measure.regionprops(label_stack)
    del label_stack #liberate memory
    
    index_list = []
    for count, item in enumerate(propsa):
        if len(item.coords) < size_threshold:
            index_list.append(count)
            
    print('Blobs that are removed:', len(index_list))
    
    stack_ = np.copy(stack)
    for i in range(len(index_list)):
        stack_ = set_to_zero(stack_, propsa[index_list[i]].coords)
                
    return stack_

In [ ]:
%%time 
skel_stack = small_skeletons_3d(img_stack)
skel_stack[skel_stack > 0] = 1
skel_stack = skel_stack.view(dtype=np.uint32)
trimmed_skel_stack = cc3d.dust(skel_stack,threshold=100, in_place=False)

In [ ]:
trimmed_test = trimmed_skel_stack[trimmed_skel_stack!=0]

In [ ]:
skel_stack = small_skeletons_3d(img_stack)
trimmed_skel_stack = trim_small_elements(skel_stack, 100)

In [ ]:
trim_test_2 = trimmed_skel_stack[trimmed_skel_stack!=0]
len(trim_test_2)

In [ ]:
from_vol_to_folder(skel_stack, r'Al207s-dense.tif', r'/data/Lucas/Substacks/AL207s/seg_substacks/skel-Al207s-dense')

In [ ]:
probability_densities = compute_skel_probas(skel_stack)

In [ ]:
probability_densities_trimmed = compute_skel_probas(trimmed_skel_stack)